In [1]:
# !pip install pyarrow

## Importing required libraries

In [2]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

#### We'll use the same NYC taxi dataset "Yellow Taxi Trip Records". Download the data for January and February 2023. Read the data for January. How many columns are there?

In [3]:
df1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df2 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [4]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
df2.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [6]:
print(f"No of columns in dataframe:{len(df1.columns)}")

No of columns in dataframe:19


## Q2. Computing duration

#### Now let's compute the duration variable. It should contain the duration of a ride in minutes. What's the standard deviation of the trips duration in January?

#### lets write compute_duration function to compute the duration in minutes for the dataframe ingested

In [7]:
def compute_duration(df):
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    return df

#### lets compute duration for training dataframe

In [8]:
df_train = compute_duration(df1)

#### lets compute standard_deviation for train_data duration variable

In [9]:
print(f"Standard_deviation for duration variable of Train data:{df_train['duration'].std():.2f}")

Standard_deviation for duration variable of Train data:42.59


## Q3. Dropping outliers

#### Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).What fraction of the records left after you dropped the outliers?

In [10]:
def filter_df(df):
    df = df[(df['duration']>=1)&(df['duration']<=60)].copy()
    return df

In [11]:
original_count = len(df_train)
df_filtered = filter_df(df_train)
filtered_count = len(df_filtered)
fraction_left = filtered_count/original_count
print(f"Fraction of the records left after dropping the outliers:{fraction_left * 100:.2f}%")

Fraction of the records left after dropping the outliers:98.12%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [12]:
def fit_dictvec(df):
    df['PULocationID'] = df['PULocationID'].astype(str)
    df['DOLocationID'] = df['DOLocationID'].astype(str)
    dicto = df[["PULocationID", "DOLocationID"]].to_dict(orient='records')
    return dicto

In [13]:
dv = DictVectorizer()

In [14]:
dicto = fit_dictvec(df_filtered)
X = dv.fit_transform(dicto)
print(f"Feature matrix shape of train data: {X.shape}")

Feature matrix shape of train data: (3009173, 515)


In [15]:
print(f"No of Feature columns: {X.shape[1]}")

No of Feature columns: 515


## Q5. Training a model

#### Now let's use the feature matrix from the previous step to train a model.Train a plain linear regression model with default parameters, where duration is the response variableCalculate the RMSE of the model on the training dataWhat's the RMSE on train?

In [17]:
lr = LinearRegression()

In [18]:
lr.fit(X, df_filtered['duration'])
y_pred = lr.predict(X)

In [19]:
rmse = mean_squared_error(df_filtered['duration'], y_pred, squared=False)
print(f"RMSE on train: {rmse:.2f}")

RMSE on train: 7.65


## Q6. Evaluating the model

#### Now let's apply this model to the validation dataset (February 2023).What's the RMSE on validation?

In [20]:
df_test = compute_duration(df2)

In [21]:
df_filtered_test = filter_df(df_test)
dicto2 = fit_dictvec(df_filtered_test)
X_val = dv.transform(dicto2)
y_val_pred = lr.predict(X_val)

In [22]:
rmse_val = mean_squared_error(df_filtered_test['duration'], y_val_pred, squared=False)
print(f"Validation RMSE: {rmse_val:.2f}")

Validation RMSE: 7.81
